In questo esempio verrà utilizzato il dataset Horovod

# Requisiti 

Per utilizzare questo tutorial, avrai bisogno di:
* Un account Azure con una sottoscrizione attiva.
* Un'area di lavoro Azure ML con cluster di computer.
* Un enviroment python.
* L'installazione di Azure Machine Learning Python SDK v2.

# 1. Connessione ad Azure Machine Learning Workspace

## 1.1 Importazione delle librerie necessarie


In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml import command, MpiDistribution
from azure.identity import DefaultAzureCredential

## 1.2 Configurazione e Gestione del WorkSpace
Per connettersi a un'area di lavoro, si necessita dei parametri identificativi: una sottoscrizione, un gruppo di risorse e un nome dell'area di lavoro. 
Utilizzeremo questi dettagli in **MLClient** da **azure.ai.ml** per ottenere una gestione dell'area di lavoro Azure Machine Learning richiesta.
Per questo tutorial utilizzeremo l'autenticazione predefinita. 

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# Configurazione ed Esecuzione dei Comandi

## 2.1 Configurazione dei comandi

**command** consente all'utente di configurare i seguenti aspetti chiave:
* **code** - Questo è il percorso in cui si trova il codice per eseguire il comando.
* **command** - Questo è il comando che deve essere eseguito. 
* **inputs** - Questo è il dizionario degli input che utilizzano le coppie di valori dei nomi per il comando. La chiave è un nome per l'input nel contesto del processo e il valore è il valore di input. Gli input possono essere referenziati nel comando utilizzando l'espressione ${{inputs.<input_name>}} . Per usare file o cartelle come input, possiamo usare la classe Input. La classe Input supporta tre parametri:
    * **type** - Il tipo dell' input. Questo può essere un uri_file o uri_folder. Il valore predefinito è uri_folder.
    * **path** - Percorso del file o della cartella. Questi possono essere file o cartelle locali o remoti. Per i file remoti  Azure supporta  http/https, wasb.
        * I **dati/dataset** o **l'archivio dati** di Azure ML sono di tipo uri_folder. Per utilizzare i dati/dataset come input, è possibile utilizzare il dataset registrato nell'area di lavoro utilizzando il formato '<nome_data>:'. Ad esempio Input(type='uri_folder', path='my_dataset:1').
    * **mode** - Modalità di come i dati devono essere consegnati alla destinazione di elaborazione. I valori consentiti sono ro_mount, rw_mount e download. Il valore predefinito è ro_mount.
* **enviroment** - Questo è l'ambiente necessario per l'esecuzione del comando. È possibile utilizzare ambienti curati o personalizzati dall'area di lavoro. Oppure può essere creato e utilizzato anche un ambiente personalizzato.
* **compute** - Il calcolo su cui verrà eseguito il comando. In questo esempio viene utilizzato un calcolo chiamato **cpu-cluster** presente nell'area di lavoro. È possibile sostituirlo in qualsiasi altro calcolo nell'area di lavoro. Lo si può eseguire sulla macchina locale usando **local** per il calcolo. Questo eseguirà il comando sul computer locale e tutti i dettagli di esecuzione e l'output del processo verranno caricati nell'area di lavoro di Azure ML.
* **distribution** - Configurazione di distribuzione per scenari di formazione distribuiti. Azure Machine Learning supporta la formazione distribuita basata su PyTorch, TensorFlow e MPI. I valori consentiti sono **PyTorch**, **TensorFlow** e **MPI**.
* **dispay_name** - Il nome del Display del lavoro.
* **description** - La descrzione dell'esperimento.

Nell'esempio seguente vedremo la distribuzione **MPI**.

In [ ]:
from azure.ai.ml import command, MpiDistribution

job = command(
    code="./src",
    command="python train.py --epochs ${{inputs.epochs}}",
    inputs={"epochs": 1},
    environment="AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu@latest",
    compute="gpu-cluster",
    instance_count=2,
    distribution=MpiDistribution(process_count_per_instance=2),
    display_name="tensorflow-mnist-distributed-horovod-example"
   )

## 2.2 Esecuzione del comando 

Utilizzando il **MLClient** creato in precedenza, si potrà eseguire il comando come processo nell'area di lavoro.

In [ ]:
returned_job = ml_client.create_or_update(job)